In [ ]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import cv2; print('opencv', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings

In [ ]:
sys.path.extend(['./models'])
print('sys.path', sys.path)

In [ ]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [ ]:
model_name = 'densenet2d'
dataset_name = 'cifar100'
unique_id = int(time.time())

img_rows, img_cols, channels = 224, 224, 3
num_classes = 100

batch_size = 32
epochs = 999

load_weights = False
fine_tuning = False

In [ ]:
from models.se_densenet import preprocess_input, SEDenseNet, SEDenseNetImageNet201

base_model = SEDenseNetImageNet201(input_shape=(img_rows, img_cols, channels),
                                  bottleneck=True,
                                  reduction=0.5,
                                  dropout_rate=0.2,
                                  weight_decay=1e-4,
                                  include_top=False,
                                  weights=None,
                                  input_tensor=None,
                                  classes=num_classes,
                                  activation='softmax')

x = base_model.output
x = keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

# model = SEDenseNet(input_shape=None,
#                    depth=40,
#                    nb_dense_block=3,
#                    growth_rate=12,
#                    nb_filter=-1,
#                    nb_layers_per_block=-1,
#                    bottleneck=False,
#                    reduction=0.0,
#                    dropout_rate=0.0,
#                    weight_decay=1e-4,
#                    subsample_initial_block=False,
#                    include_top=True,
#                    weights=None,
#                    input_tensor=None,
#                    classes=10,
#                    activation='softmax')

model.summary()
                                                                          

In [ ]:
print(model_name, len([l.name for l in model.layers if ('conv' in str(type(l)) or 'dense' in str(type(l)))])+1)

In [ ]:
from tensorflow.keras.datasets import cifar10, cifar100

(X_train, Y_train), (X_valid, Y_valid) = cifar100.load_data()

X_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in X_train])
X_valid = np.array([cv2.resize(img, (img_rows, img_cols)) for img in X_valid])

X_train = preprocess_input(X_train.astype('float'))
X_valid = preprocess_input(X_valid.astype('float'))

Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_valid = keras.utils.to_categorical(Y_valid, num_classes)

X_test = X_valid[5000:]
Y_test = Y_valid[5000:]

X_valid = X_valid[:5000]
Y_valid = Y_valid[:5000]

print('trainset:', type(X_train), X_train.shape, type(Y_train), Y_train.shape)
print('valset:', type(X_valid), X_valid.shape, type(Y_valid), Y_valid.shape)
print('testset:', type(X_test), X_test.shape, type(Y_test), Y_test.shape)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

output_dir = f'./output/{model_name}/{unique_id}/'
print('output:', output_dir)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

checkpoint = ModelCheckpoint(filepath=output_dir + "weights-e{epoch:03d}-{acc:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             save_freq='epoch',
                             load_weights_on_restart=False)

checkpoint_best = ModelCheckpoint(filepath=output_dir + "model-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=False,
                                  mode='auto', 
                                  save_freq='epoch',
                                  load_weights_on_restart=False)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=1e-8, cooldown=0, min_lr=0)
tensorBoard = TensorBoard(log_dir=output_dir + 'logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
csvLogger = CSVLogger(filename=output_dir + "train_logs.csv", separator=',', append=False)

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=(X_valid, Y_valid),
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

In [ ]:
from tensorflow.keras.models import load_model
model_trained = load_model(output_dir + "model-best.hd5")

In [ ]:
# Make predictions
predictions_test = model_trained.predict(X_test, batch_size=batch_size, verbose=1)
print(Y_test.shape, predictions_test.shape)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
score_loss = log_loss(Y_test, predictions_test)
score_acc = accuracy_score(np.argmax(Y_test, axis=1), np.argmax(predictions_test, axis=1))
print('loss:', score_loss, 'accuracy:', score_acc)